In [23]:
import zipfile
from lxml import etree

def get_epub_info(fname):
    ns = {
        'n':'urn:oasis:names:tc:opendocument:xmlns:container',
        'pkg':'http://www.idpf.org/2007/opf',
        'dc':'http://purl.org/dc/elements/1.1/'
    }

    # prepare to read from the .epub file
    zip = zipfile.ZipFile(fname)

    # find the contents metafile
    txt = zip.read('META-INF/container.xml')
    tree = etree.fromstring(txt)
    cfname = tree.xpath('n:rootfiles/n:rootfile/@full-path',namespaces=ns)[0]

    # grab the metadata block from the contents metafile
    cf = zip.read(cfname)
    tree = etree.fromstring(cf)
    p = tree.xpath('/pkg:package/pkg:metadata',namespaces=ns)[0]

    # repackage the data
    res = {}
    for s in ['title','language','creator','date','identifier']:
        res[s] = p.xpath('dc:%s/text()'%(s),namespaces=ns)[0]

    return res



In [24]:
ebook ='/home/jaimevalero/libros/Mahoma - Cesar Vidal.epub'
get_epub_info(ebook)

{'title': 'Mahoma',
 'language': 'es',
 'creator': 'César Vidal',
 'date': '2015-06-05',
 'identifier': 'urn:uuid:2ecda7b7-469e-4b71-833c-611504904bdd'}

In [25]:
import ebooklib
from ebooklib import epub
def epub2thtml(epub_path):
    book = epub.read_epub(epub_path)
    chapters = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters.append(item.get_content())
    return chapters
#epub2thtml(ebook)    

In [26]:
from bs4 import BeautifulSoup
blacklist = [   '[document]',   'noscript', 'header',   'html', 'meta', 'head','input', 'script',  "style" ]
# there may be more elements you don't want, such as "style", etc.
def chap2text(chap):
    output = ''
    soup = BeautifulSoup(chap, 'html.parser')
    text = soup.find_all(text=True)
    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    return output

In [27]:
def thtml2ttext(thtml):
    Output = []
    for html in thtml:
        text =  chap2text(html)
        Output.append(text)
    return Output

In [28]:
def epub2text(epub_path):
    chapters = epub2thtml(epub_path)
    ttext = thtml2ttext(chapters)
    return ttext

In [29]:
#out=epub2text(ebook)
#out[3]

'\n Título original:  Mahoma \n César Vidal, 2015 \n Editor digital: turolero \n ePub base r1.2 \n \n \n '

In [30]:
def filter_nonprintable(text):
    import itertools
    # Use characters of control category
    nonprintable = itertools.chain(range(0x00,0x20),range(0x7f,0xa0))
    # Use translate to remove all non-printable characters
    return text.translate({character:None for character in nonprintable})
def epub_to_clean_text(ebook):
    """
    Convert an epub to text
    Arguments:
    ----------
    ebook: Path to the file
    """
    out = epub2text(ebook)
    text=""
    for chapter in out : 
        text += filter_nonprintable(chapter)    
    return text
epub_to_clean_text(ebook)   

"  Considerado como el último profeta de la Historia por más de mil millones de personas, Mahoma sigue siendo un personaje controvertido y de innegable actualidad. La presente biografía —la más completa en español sobre el tema— realiza un exhaustivo análisis de las fuentes sobre el fundador del islam así como de las últimas investigaciones en áreas como la filología, la arqueología o la Historia. Su lectura es imprescindible para todo aquel que desee conocer a Mahoma y el islam sin prejuicios y con una base historiográfica sólida.    César Vidal  Mahoma  ePub r1.0  turolero  14.08.15   Título original:  Mahoma  César Vidal, 2015  Editor digital: turolero  ePub base r1.2       CÉSAR VIDAL MANZANARES (Madrid, 1958) es doctor en historia, filosofía y teología, así como licenciado en derecho. Ha enseñado en distintas universidades de Europa y América, y es miembro de prestigiosas instituciones académicas, como la American Society of Oriental Research o el Oriental Institute de Chicago. Ac

In [40]:
ebook

'/home/jaimevalero/libros/Mahoma - Cesar Vidal.epub'